In [6]:
val sparkDummy = spark
import sparkDummy.implicits._

# Comparison with DStream

The old spark streaming API used discretized streams or DStreams.  The model was that rather than a single, ever-growing dataset, a discretized stream provided an RDD at each trigger.  In a sense, it was a series of discretized batch operations.

![](images/streaming-flow.png)

<!--- Images from http://spark.apache.org/docs/latest/streaming-programming-guide.html --->

The model had several drawbacks:
1. DStream made it hard to deal with late or out of order data because the DStream was just discretized batches so it was difficult to update an old batch.
1. The API for DStream was very different from the API for RDDs because their underlying data model was different.
1. The streaming followed-by batch semantics made reliability hard.  If one step failed, the semantics of rerunning were unclear.

Nonetheless, it is still a more built out API with great feature support and will still be necessary for streaming applications in the near term.

# Custom Receivers in DStream

DStreams have support for custom receivers (i.e. sources).  For example, we are able to create a timed file source, which uses a separate thread to read data from a file and transmit the contents line by line, one each second.

In [3]:
import scala.io.Source
import org.apache.spark.streaming.receiver.Receiver
import org.apache.spark.storage.StorageLevel

// Should be a class ...
def timedFileSource(fileName: String) = {
    new Receiver[String](StorageLevel.MEMORY_AND_DISK_2) {
        def onStart() {
            new Thread("Timed File Source") {
                override def run() { receive() }
            }.start()
        }

        def onStop() { }

        private def receive() {
            for (line <- Source.fromFile(fileName).getLines) {
                println(line)  // print for debugging
                store(line)  // send the line as a source
                Thread.sleep(1000L)  // Wait one second
            }
        }
    }
}

# Iterative Shakespearean Wordcount using DStream

To illustrate the difference in APIs, we will write wordcount using DStream.  As we'll see, the syntax appears similar but the semantics are vastly different.
![dstream-ops](images/streaming-dstream.png)

Because each discretized RDD is its own entity, we are only see words in each time period and the wordcount will be per period, not the cumulative wordcount.
![dstream-ops](images/streaming-dstream-ops.png)

<!--- Images from http://spark.apache.org/docs/latest/streaming-programming-guide.html --->

In [4]:
import org.apache.spark._
import org.apache.spark.streaming._

val ssc = new StreamingContext(sc, Seconds(1))

In [5]:
(ssc.receiverStream(timedFileSource("data/summer.txt"))
    .flatMap(_.split(" "))
    .map(word => (word, 1))
    .reduceByKey(_ + _)
    .print())

ssc.start()             // Start the computation

-------------------------------------------
Time: 1474647881000 ms
-------------------------------------------

-------------------------------------------
Time: 1474647882000 ms
-------------------------------------------
(Shall,1)
(thee,1)
(compare,1)
(a,1)
(I,1)
(to,1)
(day?,1)
(summer’s,1)

-------------------------------------------
Time: 1474647883000 ms
-------------------------------------------
(Thou,1)
(lovely,1)
(temperate.,1)
(more,2)
(and,1)
(art,1)

-------------------------------------------
Time: 1474647884000 ms
-------------------------------------------
(shake,1)
(buds,1)
(winds,1)
(do,1)
(May,,1)
(of,1)
(darling,1)
(Rough,1)
(the,1)

-------------------------------------------
Time: 1474647885000 ms
-------------------------------------------
(lease,1)
(short,1)
(date.,1)
(too,1)
(And,1)
(a,1)
(all,1)
(hath,1)
(summer’s,1)

-------------------------------------------
Time: 1474647886000 ms
-------------------------------------------
(shines,,1)
(too,1)
(heaven,1)
(e

# Cumulative Shakespearean Wordcount using DStream

The above only gives the iterative wordcount so how do we get the cumulative wordcount?  It turns out we need to keep track of state using separate `State` variables and manually update those states in calls to `mapWithState`.  The API is definitely more involved and more cumbersome.

In [5]:
// You cannot create jobs in a started streaming context
// Consider restarting the kernel

def updateFn(key: String, value: Option[Int], state: State[Int]) = {
    // update state (stateful!)
    state.update(value.getOrElse(0) + state.getOption.getOrElse(0))

    // result to return
    (key, state.get)
}

val spec = StateSpec.function(updateFn _)

// checkpointing is mandatory
ssc.checkpoint("_checkpoints")

(ssc.receiverStream(timedFileSource("data/summer.txt"))
    .flatMap(_.split(" "))
    .map(word => (word, 1))
    .mapWithState(spec)
    .print())

ssc.start()             // Start the computation

Name: Unknown
Message: Unable to retrieve error!
StackTrace: 

# Benefits of Tungsten

We've seen the surface-level API differences between DStream and Structured Streaming, there are also many differences under the hood.  These have been added as newer versions of Spark for Datasets and Structured Streaming and are sometimes retrofitted to RDDs.

While Spark has traditionally been focused on optimizing for inter-computer network IO Efficiency.  This was the major bottle neck in mapreduce and other traditional distributed computing systems.  At this point, Spark is now intra-computer memory and CPU bound.

1. **Customized Memory Management:** Spark understands its own memory allocation needs better than the generic JVM garbage collector.  Tungsten takes advantage of this by using `sun.misc.Unsafe`, which exposes C-Style off-heap memory access.  The result is fewer unecessary garbage colleciton events and improved performance.
1. **Binary Encoding:** Spark traditionally needed to serialized data to JVM objects, it now uses the Tungsten binary encoding.  This has two major advantages.  First, it decreases the memory footprint.  While "ABCD" would take 4 bytes of UTF-8 to encode, it would be stored using 48 bytes as a JVM object.  Second, instead of serializing into objects, it's able to perform many actions directly on the the raw binary encoding, reducing the computational overhead for serialization / deserialization.
1. **Code Generation:** Spark historically used generic JVM function evaluation.  Given the virtual function lookups, automated boxing of primitive types, and other JVM overhead, this dramatically slows down the computation.  By using type information, Tungsten is able to generate byte code and speed up performance.
1. **Cache-aware Computation:** Tungsten lays out its memory in a way that takes advantage of CPU cache locality to reduce cache spilling and speed up computations.

For more information, check out [this blog post](https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html) or [this presentation](http://www.slideshare.net/SparkSummit/deep-dive-into-project-tungsten-josh-rosen).

# Tungsten Performance Benefit

The following two operations count a million integers in memory.
- The first uses RDDs and must serialize all the objects into Java Objects.
- The second use sTungsten and uses a more compact binary encoding.

Tungsten saves a factor of nearly 4 on memory:

![Tungsten Memory](images/Tungsten_Memory.png)

To reproduce the example, run the following code and goto your Spark UI Viewer.  You can launch a new shell using `make spark-shell` (recommended), this will be at [http://localhost:5050/storage/](http://localhost:5050/storage/).  The default Spark UI port is 4040.

In [4]:
val million = 0 until math.pow(10, 6).toInt

// Using RDDs
sc.parallelize(million).cache.count

// Using Tungsten
million.toDF.cache.count

1000000

# Benefits of Catalyst

Optimizations include:
1. **Constant Folding:** Evaluating constant expressions at compile time rather than at run time.
1. **Predicate Pushdown:** Running operations that reduce the dataload (e.g. selecting columns or filtering rows) earlier in the query.  This reduces the amount of data that needs to be.
1. **Boolean Expression Simplification:** Simplifies boolean expressions.
1. **Projection Pruning:** Only read the columns (fields) used in a computation.
1. **Pipelining Operations:** Combines multiple projection and filter operations into a single map operation.
1. **Cost-based Optimization:** Spark actually builds multiple plans to compute a query, computes their cost, and chooses the cheapest one.
1. **Code Generation:** The final operational plan is transformed into optimized Java bytecode.